## Data
Being able to easily load and process data is a crucial task that can make any data science more pleasant. In this notebook, we will cover most common types often encountered in data science tasks, and we will be using this data throughout the rest of this tutorial.

In [1]:
using BenchmarkTools
using DataFrames
using DelimitedFiles
using CSV
using XLSX
using Downloads

# 🗃️ Get some data
In Julia, it's pretty easy to dowload a file from the web using the `download` (https://docs.julialang.org/en/v1/stdlib/Downloads/) function. But also, you can use your favorite command line commad to download files by easily switching from Julia via the `;` key. Let's try both.

Note: `download` depends on external tools such as curl, wget or fetch. So you must have one of these.

In [2]:
P = Downloads.download("https://raw.githubusercontent.com/nassarhuda/easy_data/master/programming_languages.csv",
    "programming_languages.csv")

"programming_languages.csv"

Another way would be to use a shell command to get the same file.

# 📂 Read your data from text files.
The key question here is to load data from files such as `csv` files, `xlsx` files, or just raw text files. We will go over some Julia packages that will allow us to read such files very easily.

Let's start with the package `DelimitedFiles` which is in the standard library.

In [3]:
#=
readdlm(source, 
    delim::AbstractChar, 
    T::Type, 
    eol::AbstractChar; 
    header=false, 
    skipstart=0, 
    skipblanks=true, 
    use_mmap, 
    quotes=true, 
    dims, 
    comments=false, 
    comment_char='#')
=#
P,H = readdlm("programming_languages.csv",',';header=true);

In [4]:
# To write to a text file, you can:
writedlm("programminglanguages_dlm.txt", P, '-')

A more powerful package to use here is the `CSV` package. By default, the CSV package imports the data to a DataFrame, which can have several advantages as we will see below.

In general,[`CSV.jl`](https://juliadata.github.io/CSV.jl/stable/) is the recommended way to load CSVs in Julia. Only use `DelimitedFiles` when you have a more complicated file where you want to specify several things.

In [5]:
C = CSV.read("programming_languages.csv", DataFrame);

In [6]:
@show typeof(C)
C[1:10,:]
# C.year #[!,:year]

typeof(C) = DataFrame


Row,year,language
,Int64,String31
1,1951,Regional Assembly Language
2,1952,Autocode
3,1954,IPL
4,1955,FLOW-MATIC
5,1957,FORTRAN
6,1957,COMTRAN
7,1958,LISP
8,1958,ALGOL 58
9,1959,FACT


In [7]:
@show typeof(P)
P[1:10,:]

typeof(P) = Matrix{Any}


10×2 Matrix{Any}:
 1951  "Regional Assembly Language"
 1952  "Autocode"
 1954  "IPL"
 1955  "FLOW-MATIC"
 1957  "FORTRAN"
 1957  "COMTRAN"
 1958  "LISP"
 1958  "ALGOL 58"
 1959  "FACT"
 1959  "COBOL"

In [8]:
names(C)

2-element Vector{String}:
 "year"
 "language"

In [9]:
describe(C)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,year,1982.99,1951,1986.0,2014,0,Int64
2,language,,ALGOL 58,,dBase III,0,String31


In [10]:
@btime P,H = readdlm("programming_languages.csv",',';header=true);
@btime C = CSV.read("programming_languages.csv", DataFrame);

  30.358 μs (328 allocations: 50.72 KiB)
  39.069 μs (472 allocations: 47.32 KiB)


In [11]:
# To write to a *.csv file using the CSV package
CSV.write("programminglanguages_CSV.csv", DataFrame(P, :auto))

"programminglanguages_CSV.csv"

Another type of files that we may often need to read is `XLSX` files. Let's try to read a new file.

In [12]:
T = XLSX.readdata("data/zillow_data_download_april2020.xlsx", #file name
    "Sale_counts_city", #sheet name
    "A1:F9" #cell range
    )

9×6 Matrix{Any}:
      "RegionID"  "RegionName"    …      "2008-03"      "2008-04"
  6181            "New York"             missing        missing
 12447            "Los Angeles"      1446           1705
 39051            "Houston"          2926           3121
 17426            "Chicago"          2910           3022
  6915            "San Antonio"   …  1479           1529
 13271            "Philadelphia"     1609           1795
 40326            "Phoenix"          1310           1519
 18959            "Las Vegas"        1618           1856

If you don't want to specify cell ranges... though this will take a little longer...

In [13]:
G = XLSX.readtable("data/zillow_data_download_april2020.xlsx","Sale_counts_city");

In [14]:
G.data[1][1:10]

10-element Vector{Any}:
  6181
 12447
 39051
 17426
  6915
 13271
 40326
 18959
 54296
 38128

And we can easily store this data in a DataFrame. `DataFrame(G...)` uses the "splat" operator to _unwrap_ these arrays and pass them to the DataFrame constructor.

In [15]:
D = DataFrame(G)
D |> nrow

28759

In [16]:
foods = ["apple", "cucumber", "tomato", "banana"]
calories = [105,47,22,105]
prices = [0.85,1.6,0.8,0.6,]
dataframe_calories = DataFrame(item=foods,calories=calories)
dataframe_prices = DataFrame(item=foods,price=prices)

Row,item,price
,String,Float64
1,apple,0.85
2,cucumber,1.6
3,tomato,0.8
4,banana,0.6


In [17]:
DF = innerjoin(dataframe_calories,dataframe_prices,on=:item)

Row,item,calories,price
,String,Int64,Float64
1,apple,105,0.85
2,cucumber,47,1.6
3,tomato,22,0.8
4,banana,105,0.6


In [18]:
# we can also use the DataFrame constructor on a Matrix
DataFrame(T, :auto)

Row,x1,x2,x3,x4,x5,x6
,Any,Any,Any,Any,Any,Any
1,RegionID,RegionName,StateName,SizeRank,2008-03,2008-04
2,6181,New York,New York,1,missing,missing
3,12447,Los Angeles,California,2,1446,1705
4,39051,Houston,Texas,3,2926,3121
5,17426,Chicago,Illinois,4,2910,3022
6,6915,San Antonio,Texas,5,1479,1529
7,13271,Philadelphia,Pennsylvania,6,1609,1795
8,40326,Phoenix,Arizona,7,1310,1519
9,18959,Las Vegas,Nevada,8,1618,1856


You can also easily write data to an XLSX file

In [19]:
XLSX.writetable("writefile_using.xlsx",G)

## ⬇️ Importing your data

Often, the data you want to import is not stored in plain text, and you might want to import different kinds of types. Here we will go over importing `jld`, `npz`, `rda`, and `mat` files. Hopefully, these four will capture the types from four common programming languages used in Data Science (Julia, Python, R, Matlab).

We will use a toy example here of a very small matrix. But the same syntax will hold for bigger files.

```
4×5 Array{Int64,2}:
 2  1446  1705  1795  1890
 3  2926  3121  3220  3405
 4  2910  3022  2937  3224
 5  1479  1529  1582  1761
 ```

In [20]:
using JLD
jld_data = JLD.load("data/mytempdata.jld")
save("mywrite.jld", "A", jld_data)

In [21]:
using NPZ
npz_data = npzread("data/mytempdata.npz")
npzwrite("mywrite.npz", npz_data)

In [22]:
using RData
R_data = RData.load("data/mytempdata.rda")
using RCall
@rput R_data
R"save(R_data, file=\"mywrite.rda\")"

RObject{NilSxp}
NULL


In [23]:
using MAT
Matlab_data = matread("data/mytempdata.mat")
matwrite("mywrite.mat",Matlab_data)

In [24]:
@show typeof(jld_data)
@show typeof(npz_data)
@show typeof(R_data)
@show typeof(Matlab_data)
;

typeof(jld_data) = Dict{String, Any}
typeof(npz_data) = Matrix{Int64}
typeof(R_data) = Dict{String, Any}
typeof(Matlab_data) = Dict{String, Any}


In [25]:
Matlab_data

Dict{String, Any} with 1 entry:
  "tempdata" => [2 1446 … 1795 1890; 3 2926 … 3220 3405; 4 2910 … 2937 3224; 5 …

# 🔢 Time to process the data from Julia
We will mainly cover `Matrix` (or `Vector`), `DataFrame`s, and `dict`s (or dictionaries). Let's bring back our programming languages dataset and start playing it using the matrix it's stored in.

In [26]:
P[1:10, :]

10×2 Matrix{Any}:
 1951  "Regional Assembly Language"
 1952  "Autocode"
 1954  "IPL"
 1955  "FLOW-MATIC"
 1957  "FORTRAN"
 1957  "COMTRAN"
 1958  "LISP"
 1958  "ALGOL 58"
 1959  "FACT"
 1959  "COBOL"

Here are some quick questions we might want to ask about this simple data.
- Which year was was a given language invented?
- How many languages were created in a given year?

In [27]:
function year_created(P, language::String)
  idx = findfirst(==(language), P[:, 2])
  isnothing(idx) && return nothing
  P[idx, 1]
end

year_created (generic function with 1 method)

In [28]:
# Q1: Which year was was a given language invented?
println(year_created(P, "Julia"))
println(year_created(P, "W"))

2012
nothing


In [29]:
# Q2: How many languages were created in a given year?
function how_many_per_year(P,year::Int64)
    year_count = length(findall(P[:,1].==year))
    return year_count
end
how_many_per_year(P,2011)

4

Now let's try to store this data in a DataFrame...

In [30]:
P_df = C

Row,year,language
,Int64,String31
1,1951,Regional Assembly Language
2,1952,Autocode
3,1954,IPL
4,1955,FLOW-MATIC
5,1957,FORTRAN
6,1957,COMTRAN
7,1958,LISP
8,1958,ALGOL 58
9,1959,FACT


And now let's answer the same questions we just answered...

In [31]:
try
  println(year_created(P, "Julia"))
  println(year_created(P, "W"))
catch e
  if e isa KeyError
    println("Language not found: $(e.key)")
  end
end

2012
nothing


In [32]:
how_many_per_year(P_df,2011)

4

Next, we'll use dictionaries. A quick way to create a dictionary is with the `Dict()` command. But this creates a dictionary without types. Here, we will specify the types of this dictionary.

In [33]:
# A quick example to show how to build a dictionary
Dict([("A", 1), ("B", 2),(1,[1,2])])

Dict{Any, Any} with 3 entries:
  "B" => 2
  "A" => 1
  1   => [1, 2]

In [34]:
P_dict = Dict{Integer,Vector{String}}()

Dict{Integer, Vector{String}}()

Now, let's populate the dictionary with years as keys and vectors that hold all the programming languages created in each year as their values. Even though this looks like more work, we often need to do it just once.

In [35]:
names(P_df)
dict = Dict(k.year => collect(v.language) for (k, v) in pairs(groupby(P_df, :year)))

Dict{Int64, Vector{String31}} with 45 entries:
  1985 => ["Eiffel"]
  2002 => ["Scratch"]
  1952 => ["Autocode"]
  1963 => ["CPL"]
  1964 => ["Speakeasy", "BASIC", "PL/I"]
  1967 => ["BCPL"]
  2001 => ["C#", "D"]
  1991 => ["Python", "Visual Basic"]
  1957 => ["FORTRAN", "COMTRAN"]
  1988 => ["Tcl", "Wolfram Language "]
  1955 => ["FLOW-MATIC"]
  1951 => ["Regional Assembly Language"]
  1994 => ["CLOS "]
  2011 => ["Dart", "Kotlin", "Red", "Elixir"]
  1959 => ["FACT", "COBOL", "RPG"]
  1962 => ["APL", "Simula", "SNOBOL"]
  2005 => ["F#"]
  1969 => ["B"]
  1972 => ["C", "Smalltalk", "Prolog"]
  1997 => ["Rebol"]
  1986 => ["Objective-C", "LabVIEW ", "Erlang"]
  1993 => ["Lua", "R"]
  1958 => ["LISP", "ALGOL 58"]
  1987 => ["Perl"]
  1954 => ["IPL"]
  ⋮    => ⋮

In [36]:
length(keys(dict))

45

In [37]:
length(unique(P[:,1]))

45

In [38]:
function year_created(dict, language::String)
    sorted_years = dict |> keys |> collect |> sort
    idx = findfirst(year -> language in dict[year], sorted_years)
    isnothing(idx) ? nothing : sorted_years[idx]
end
year_created(dict, "Julia")

2012

In [39]:
# Q2: How many languages were created in a given year?
how_many_per_year(P_dictionary,year::Int64) = length(P_dictionary[year])
how_many_per_year(dict,2011)

4

# 📝 A note about missing data

In [40]:
# assume there were missing values in our dataframe
P[1,1] = missing
P_df = DataFrame(year = P[:,1], language = P[:,2])
P_df |> nrow

73

In [41]:
P_df |> dropmissing |> nrow

72